In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial import distance
import math
import os
import glob

from PIL import Image, ImageOps, ImageEnhance
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.morphology import watershed
import ast

In [10]:
def ConstrastStretching(I):
    #get the minimal pixel value
    c = np.amin(I)
    #get the maxmial pixel value
    d = np.amax(I)
    a = 0
    b = 255
    #given funcition
    O = (I-c)*((b-a)/(d-c))+a
    #transform the datatype into uint8
    O = O.astype('uint8')
    return O

def read_img(path):
    sequence_tif = {}
    norm_seq_tif = {}
    img_array = []
    files = glob.glob(path)
    files.sort()
    for file in files:
        tmp_img = cv2.imread(file)
        img_array.append(tmp_img)
    return img_array

def normalize(image):
    img = image.copy().astype(np.float32)
    img -= np.mean(img)
    img /= np.linalg.norm(img)
    img = np.clip(img, 0, 255)
    img *= (1./float(img.max()))
    return (img*255).astype(np.uint8)

In [11]:
def draw_trajectory(imgs, centroids, rects):
    #image_a = np.zeros((imgs[0].shape[0], imgs[0].shape[1]))
    image_a = image_a = imgs[-1]
    for i in range(1, len(imgs)):
        last_img=imgs[i-1]
        cur_img=imgs[i]
        last_centro = centroids[i-1]
        cur_centro = centroids[i]
#         last_rect = rects[i-1]
#         cur_rect = rects[i]
        for k in last_centro: # pt in last pic
            dis = []
            for n in cur_centro:
                dis.append(distence(k,n))
            sorted_dis = sorted(dis)
            pt1=cur_centro[dis.index(sorted_dis[0])] # the closest pt to k
            pt2=cur_centro[dis.index(sorted_dis[1])] # the scond closest pt to k
            
            ind1=cur_centro.index(pt1)
            ind2=cur_centro.index(pt2)
            indk=last_centro.index(k)
            ind = [ind1, ind2]
            # area: ((up_left), (down_right))
#             ao = ((last_rect[indk][0], last_rect[indk][1]), (last_rect[indk][0]+last_rect[indk][2], last_rect[indk][1]+last_rect[indk][3]))
#             a1 = ((cur_rect[ind1][0], cur_rect[ind1][1]), (cur_rect[ind1][0]+cur_rect[ind1][2], cur_rect[ind1][1]+cur_rect[ind1][3]))
#             a2 = ((cur_rect[ind2][0], cur_rect[ind2][1]), (cur_rect[ind2][0]+cur_rect[ind2][2], cur_rect[ind2][1]+cur_rect[ind2][3]))
#             diff1 = distence(ao[0], a1[0])+distence(ao[1], a1[1])
#             diff2 = distence(ao[0], a2[0])+distence(ao[1], a2[1])
            dist1 = distence(k, pt1)
            dist2 = distence(k, pt2)
#             diff = [diff1, diff2]
            dist = [dist1, dist2]
            for i in range(len(dist)):
                if dist[i]<3*min(dist)/2:
                    image_a = cv2.arrowedLine(image_a, k, cur_centro[ind[i]], (255, 255, 255), 1)
    return image_a
#  diff[i]<(last_rect[indk][2]+last_rect[indk][3]) and 
def distence(tup1, tup2):
    return abs(tup1[0]-tup2[0]) + abs(tup1[1]-tup2[1])

# pic = draw_trajectory(imgs, centro, centro_all)
def toneptwo(imgs, centroids, rects):
    pics = []
    for i in range(1, len(centroids)):
        img_to_draw = imgs[:i+1]
        centro_to_draw = centroids[:i+1]
        centroall_to_draw = rects[:i+1]
        pic = draw_trajectory(img_to_draw, centro_to_draw, centroall_to_draw)
        pics.append(pic)
        #cv2.imwrite(f"trajectory{i}.tif", pic)
    return pics

In [12]:
def mitosis_detection_2(img, thresh, contours, standard_distance):
    centers = []
    radiuses = {}
    contour_dic = {}
    for contour in contours:
        (x,y), radius = cv2.minEnclosingCircle(contour)
        center = (x, y)
        centers.append(center)
        radiuses[center] = radius
        contour_dic[center] = contour
    kernel = np.ones((3,3),np.uint8)
    tmp = cv2.erode(thresh, kernel, iterations = 4)
    _, tmp_contours, _ = cv2.findContours(tmp, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    tmp_centers = []
    for contour in tmp_contours:
        (x,y), radius = cv2.minEnclosingCircle(contour)
        center = (x, y)
        tmp_centers.append(center)
    dividing_cells = set()
    E_distances = distance.cdist(tmp_centers, tmp_centers, metric='euclidean')
    for x in range(len(E_distances)):
        for y in range(len(E_distances)):
            if E_distances[x][y] < standard_distance and x != y:
                point_1 = tmp_centers[x]
                point_2 = tmp_centers[y]
                dividing_cells.add(point_1)
                dividing_cells.add(point_2)
    #print(dividing_cells)
    dividing_centers = set()
    for center in dividing_cells:
        for x in centers:
            dist = math.sqrt((x[0]-center[0])**2 + (x[1]-center[1])**2)
            if dist < radiuses[x]:
#                 contour = contour_dic[x]
#                 cv2.drawContours(img, [contour],-1, (0,0,0), 15)
                dividing_centers.add(x)
                cv2.ellipse(img, (int(x[0]), int(x[1])), (25,25), 0, 0, 360, (125,0,0), 5)
    #print (dividing_centers)
    for x in centers:
        if x not in dividing_centers:
            cv2.ellipse(img, (int(x[0]), int(x[1])), (15,10), 0, 0, 360, (255,0,0), 5)
    return img

def Fluo(path):
    img_array = read_img(path)
    (h, w, l) = img_array[0].shape
    out = cv2.VideoWriter('project.avi', cv2.VideoWriter_fourcc(*'XVID'),5,(w, h))
    centers_list = []
    imgs = []
    x_y_w_h_s = []
    for img in img_array:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = ConstrastStretching(img)
        gblur = cv2.GaussianBlur(img, (5, 5), 0)
        ret, thresh = cv2.threshold(gblur,32,255,cv2.THRESH_BINARY)
        kernel = np.ones((4, 4),np.uint8)
        thresh = cv2.erode(thresh, kernel, iterations = 3)
        _, contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        centers = []
        tmp_centers = []
        x_y_w_h = []
        for contour in contours:
            x_y_w_h.append(cv2.boundingRect(contour))
            (x,y), radius = cv2.minEnclosingCircle(contour)
            center = (x, y)
            tmp_centers.append((int(x), int(y)))
            centers.append(center)
            cv2.drawContours(img, [contour],-1, (255, 0, 0), 2)
            #cv2.ellipse(img, (int(center[0]), int(center[1])), (15,10), 0, 0, 360, (255,0,0), 5)
        centers_list.append(tmp_centers)
        x_y_w_h_s.append(x_y_w_h)
        img = mitosis_detection_2(img, thresh, contours, 30)
        imgs.append(img)
        #print(len(img))
        #img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        #out.write(img)
    imgs = toneptwo(imgs, centers_list, [])
    for img in imgs:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        out.write(img)
    #plt.imshow(imgs[90])
    out.release()

In [13]:
path = "images/Fluo-N2DL-HeLa/Sequence 1/*.tif"
Fluo(path)

In [14]:
def mitosis_detection_3(img, thresh, contours, standard_distance):
    centers = []
    radiuses = {}
    contour_dic = {}
    for contour in contours:
        (x,y), radius = cv2.minEnclosingCircle(contour)
        center = (x, y)
        centers.append(center)
        radiuses[center] = radius
        contour_dic[center] = contour
    kernel = np.ones((2,2),np.uint8)
    tmp = cv2.erode(thresh, kernel, iterations = 5)
    _, tmp_contours, _ = cv2.findContours(tmp, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    tmp_centers = []
    for contour in tmp_contours:
        (x,y), radius = cv2.minEnclosingCircle(contour)
        center = (x, y)
        tmp_centers.append(center)
    dividing_cells = set()
    E_distances = distance.cdist(tmp_centers, tmp_centers, metric='euclidean')
    for x in range(len(E_distances)):
        for y in range(len(E_distances)):
            if E_distances[x][y] < standard_distance and x != y:
                point_1 = tmp_centers[x]
                point_2 = tmp_centers[y]
                dividing_cells.add(point_1)
                dividing_cells.add(point_2)
    dividing_centers = set()
    for center in dividing_cells:
        for x in centers:
            dist = math.sqrt((x[0]-center[0])**2 + (x[1]-center[1])**2)
            if dist < radiuses[x]:
                dividing_centers.add(x)
                cv2.ellipse(img, (int(x[0]), int(x[1])), (20,15), 0, 0, 360, (255,0,0), 2)
#                 contour = contour_dic[x]
#                 cv2.drawContours(img, [contour],-1, (125,0,0), 2)
    for x in centers:
        if x not in dividing_centers:
            contour = contour_dic[x]
            #cv2.drawContours(img, [contour],-1, (0,0,0), 2)
            cv2.ellipse(img, (int(x[0]), int(x[1])), (10,5), 0, 0, 360, (0,0,0), 2)
    return img

def PhC(path):
    img_array = read_img(path)
    (h, w, l) = img_array[0].shape
    out = cv2.VideoWriter('project1.avi', cv2.VideoWriter_fourcc(*'XVID'),5,(w, h))
    #num = 0
    for img in img_array:
    #     if num > 100:
    #         break
    #     num += 1
        img2 = img.copy()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = ConstrastStretching(img)
        kernel = np.ones((3,3),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        gblur = cv2.GaussianBlur(img, (5, 5), 0)
        ret, thresh = cv2.threshold(gblur,200,255,cv2.THRESH_BINARY)
        _, contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        centers = []
        for contour in contours:
            (x,y), radius = cv2.minEnclosingCircle(contour)
            center = (x, y)
            centers.append(center)
            #cv2.drawContours(img, [contour],-1, (255, 0, 0), 2)
            #cv2.ellipse(img, (int(center[0]), int(center[1])), (15,10), 0, 0, 360, (255,0,0), 5)
        img2 = mitosis_detection_3(img2, thresh, contours, 20)
        #img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        out.write(img2)
        #plt.imshow(img)
    out.release()

In [15]:
path = "images/PhC-C2DL-PSC/Sequence 1/*.tif"
PhC(path)